# 🧙‍♂️ IS IT YOU HARRY? - CNN Character Recognition

Ce notebook implémente un réseau de neurones convolutif (CNN) pour reconnaître **10 personnages** d'Harry Potter.

## 📋 Objectifs
1. Créer/charger un dataset d'images de personnages
2. Prétraiter les données (normalisation, augmentation)
3. Créer un modèle CNN
4. Entraîner le modèle
5. Évaluer les performances (précision, matrice de confusion)
6. Sauvegarder le modèle

## 👥 Personnages reconnus
1. Harry Potter
2. Hermione Granger
3. Ron Weasley
4. Albus Dumbledore
5. Severus Snape
6. Voldemort
7. Draco Malfoy
8. Hagrid
9. Minerva McGonagall
10. Sirius Black

## 📦 Import des bibliothèques

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Sklearn
from sklearn.metrics import classification_report, confusion_matrix

# Configuration
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## ⚙️ Configuration

In [ ]:
# Chemins
BASE_DIR = Path('../data')
TRAIN_DIR = BASE_DIR / 'train'
VAL_DIR = BASE_DIR / 'val'
TEST_DIR = BASE_DIR / 'test'
MODEL_DIR = Path('../models')

# Hyperparamètres
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.001

# Personnages (classes)
CHARACTERS = [
    'harry_potter',
    'hermione_granger',
    'ron_weasley',
    'albus_dumbledore',
    'severus_snape',
    'voldemort',
    'draco_malfoy',
    'hagrid',
    'minerva_mcgonagall',
    'sirius_black'
]

NUM_CLASSES = len(CHARACTERS)
print(f"Nombre de classes: {NUM_CLASSES}")
print(f"Classes: {CHARACTERS}")

## 📊 Création du dataset de démonstration

⚠️ **Note**: Pour une vraie utilisation, vous devez collecter ~200 images par personnage.

Ce code crée un dataset de démonstration avec des images synthétiques pour tester le pipeline.

In [ ]:
def create_demo_dataset():
    """Crée un dataset de démonstration avec des images synthétiques."""
    from PIL import Image, ImageDraw, ImageFont
    import random
    
    print("🎨 Création du dataset de démonstration...")
    
    # Nombre d'images par classe et par split
    train_samples = 140  # 70%
    val_samples = 30     # 15%
    test_samples = 30    # 15%
    
    for character in tqdm(CHARACTERS, desc="Création des personnages"):
        # Créer les dossiers
        (TRAIN_DIR / character).mkdir(parents=True, exist_ok=True)
        (VAL_DIR / character).mkdir(parents=True, exist_ok=True)
        (TEST_DIR / character).mkdir(parents=True, exist_ok=True)
        
        # Créer des images synthétiques avec des couleurs différentes par personnage
        char_color = (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
        
        # Images d'entraînement
        for i in range(train_samples):
            img = Image.new('RGB', IMG_SIZE, color=char_color)
            draw = ImageDraw.Draw(img)
            # Ajouter du bruit pour varier les images
            for _ in range(100):
                x, y = random.randint(0, IMG_SIZE[0]-1), random.randint(0, IMG_SIZE[1]-1)
                noise_color = tuple(min(255, max(0, c + random.randint(-50, 50))) for c in char_color)
                draw.point((x, y), fill=noise_color)
            img.save(TRAIN_DIR / character / f"{character}_{i:04d}.jpg")
        
        # Images de validation
        for i in range(val_samples):
            img = Image.new('RGB', IMG_SIZE, color=char_color)
            draw = ImageDraw.Draw(img)
            for _ in range(100):
                x, y = random.randint(0, IMG_SIZE[0]-1), random.randint(0, IMG_SIZE[1]-1)
                noise_color = tuple(min(255, max(0, c + random.randint(-50, 50))) for c in char_color)
                draw.point((x, y), fill=noise_color)
            img.save(VAL_DIR / character / f"{character}_{i:04d}.jpg")
        
        # Images de test
        for i in range(test_samples):
            img = Image.new('RGB', IMG_SIZE, color=char_color)
            draw = ImageDraw.Draw(img)
            for _ in range(100):
                x, y = random.randint(0, IMG_SIZE[0]-1), random.randint(0, IMG_SIZE[1]-1)
                noise_color = tuple(min(255, max(0, c + random.randint(-50, 50))) for c in char_color)
                draw.point((x, y), fill=noise_color)
            img.save(TEST_DIR / character / f"{character}_{i:04d}.jpg")
    
    print("✅ Dataset de démonstration créé avec succès!")
    print(f"   - Train: {train_samples * NUM_CLASSES} images")
    print(f"   - Val: {val_samples * NUM_CLASSES} images")
    print(f"   - Test: {test_samples * NUM_CLASSES} images")

# Créer le dataset si il n'existe pas
if not TRAIN_DIR.exists() or len(list(TRAIN_DIR.glob('*/*.jpg'))) == 0:
    create_demo_dataset()
else:
    print("✅ Dataset déjà existant")

## 🔍 Exploration du dataset

In [ ]:
def count_images(directory):
    """Compte les images par classe."""
    counts = {}
    for character in CHARACTERS:
        char_dir = directory / character
        if char_dir.exists():
            counts[character] = len(list(char_dir.glob('*.jpg')))
        else:
            counts[character] = 0
    return counts

# Compter les images
train_counts = count_images(TRAIN_DIR)
val_counts = count_images(VAL_DIR)
test_counts = count_images(TEST_DIR)

# Afficher les statistiques
print("📊 Distribution du dataset:\n")
df_stats = pd.DataFrame({
    'Train': train_counts,
    'Validation': val_counts,
    'Test': test_counts
})
df_stats['Total'] = df_stats.sum(axis=1)
print(df_stats)
print(f"\nTotal: {df_stats['Total'].sum()} images")

In [ ]:
# Visualiser quelques exemples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Exemples d\'images par personnage', fontsize=16, fontweight='bold')

for idx, character in enumerate(CHARACTERS):
    char_dir = TRAIN_DIR / character
    images = list(char_dir.glob('*.jpg'))
    if images:
        img_path = images[0]
        img = plt.imread(img_path)
        ax = axes[idx // 5, idx % 5]
        ax.imshow(img)
        ax.set_title(character.replace('_', ' ').title(), fontsize=10)
        ax.axis('off')

plt.tight_layout()
plt.show()

## 🔄 Préparation des données avec Data Augmentation

In [ ]:
# Data augmentation pour l'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Normalisation seulement pour validation et test
test_datagen = ImageDataGenerator(rescale=1./255)

# Générateurs de données
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

print(f"\nClasses détectées: {train_generator.class_indices}")
print(f"Nombre d'images d'entraînement: {train_generator.samples}")
print(f"Nombre d'images de validation: {val_generator.samples}")
print(f"Nombre d'images de test: {test_generator.samples}")

## 🏗️ Architecture du modèle CNN

In [ ]:
def create_cnn_model(input_shape=(128, 128, 3), num_classes=10):
    """
    Crée un modèle CNN simple pour la classification d'images.
    
    Architecture:
    - 3 blocs convolutifs avec pooling
    - Couches fully connected
    - Dropout pour régularisation
    """
    model = models.Sequential([
        # Bloc 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloc 2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloc 3
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloc 4
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fully Connected
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Sortie
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Créer le modèle
model = create_cnn_model(input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES)

# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Afficher l'architecture
print("\n🏗️ Architecture du modèle:\n")
model.summary()

## 🎓 Entraînement du modèle

In [ ]:
# Créer le dossier pour les modèles
MODEL_DIR.mkdir(exist_ok=True)

# Callbacks
callbacks = [
    ModelCheckpoint(
        MODEL_DIR / 'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

# Entraînement
print("\n🎓 Début de l'entraînement...\n")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Entraînement terminé!")

## 📈 Visualisation des courbes d'entraînement

In [ ]:
# Créer les graphiques
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Précision
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('Précision du modèle', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Précision')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Perte
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('Perte du modèle', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Perte')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODEL_DIR / 'training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Graphiques sauvegardés dans {MODEL_DIR / 'training_curves.png'}")

## 🧪 Évaluation sur le test set

In [ ]:
# Charger le meilleur modèle
model = keras.models.load_model(MODEL_DIR / 'best_model.h5')

# Évaluation
print("\n🧪 Évaluation sur le test set...\n")
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print(f"\n📊 Résultats finaux:")
print(f"   - Test Loss: {test_loss:.4f}")
print(f"   - Test Accuracy: {test_accuracy * 100:.2f}%")

## 📊 Matrice de confusion

In [ ]:
# Prédictions
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)

# Visualisation
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=[c.replace('_', ' ').title() for c in CHARACTERS],
    yticklabels=[c.replace('_', ' ').title() for c in CHARACTERS],
    cbar_kws={'label': 'Nombre de prédictions'}
)
plt.title('Matrice de Confusion - Reconnaissance de Personnages', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Vérité', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(MODEL_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Matrice de confusion sauvegardée dans {MODEL_DIR / 'confusion_matrix.png'}")

## 📝 Rapport de classification détaillé

In [ ]:
# Rapport de classification
class_names = [c.replace('_', ' ').title() for c in CHARACTERS]
report = classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    output_dict=True
)

# Convertir en DataFrame
df_report = pd.DataFrame(report).transpose()
print("\n📝 Rapport de classification:\n")
print(df_report.round(3))

# Sauvegarder le rapport
df_report.to_csv(MODEL_DIR / 'classification_report.csv')
print(f"\n📊 Rapport sauvegardé dans {MODEL_DIR / 'classification_report.csv'}")

## 💾 Sauvegarde du modèle final

In [ ]:
# Sauvegarder le modèle final
final_model_path = MODEL_DIR / 'character_recognition_final.h5'
model.save(final_model_path)
print(f"✅ Modèle final sauvegardé dans {final_model_path}")

# Sauvegarder les métadonnées
metadata = {
    'model_name': 'Harry Potter Character Recognition CNN',
    'num_classes': NUM_CLASSES,
    'characters': CHARACTERS,
    'img_size': IMG_SIZE,
    'test_accuracy': float(test_accuracy),
    'test_loss': float(test_loss),
    'epochs_trained': len(history.history['loss']),
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE
}

import json
with open(MODEL_DIR / 'model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Métadonnées sauvegardées dans {MODEL_DIR / 'model_metadata.json'}")

## 🎯 Analyse des erreurs

In [ ]:
# Trouver les erreurs de classification
errors = np.where(y_pred != y_true)[0]
print(f"\n❌ Nombre d'erreurs: {len(errors)} / {len(y_true)} ({len(errors)/len(y_true)*100:.2f}%)")

if len(errors) > 0:
    # Analyser les erreurs les plus fréquentes
    error_pairs = []
    for idx in errors:
        true_class = CHARACTERS[y_true[idx]]
        pred_class = CHARACTERS[y_pred[idx]]
        error_pairs.append((true_class, pred_class))
    
    # Compter les paires d'erreurs
    from collections import Counter
    error_counts = Counter(error_pairs)
    
    print("\n🔍 Erreurs les plus fréquentes:")
    for (true_class, pred_class), count in error_counts.most_common(10):
        print(f"   {true_class} → {pred_class}: {count} fois")
else:
    print("\n🎉 Aucune erreur! Modèle parfait!")

## 📊 Résumé final

In [ ]:
print("\n" + "="*70)
print("🎉 RÉSUMÉ FINAL - IS IT YOU HARRY?")
print("="*70)
print(f"\n✅ Modèle entraîné avec succès!")
print(f"\n📊 Statistiques:")
print(f"   - Nombre de personnages: {NUM_CLASSES}")
print(f"   - Images d'entraînement: {train_generator.samples}")
print(f"   - Images de validation: {val_generator.samples}")
print(f"   - Images de test: {test_generator.samples}")
print(f"   - Taille des images: {IMG_SIZE}")
print(f"\n🎯 Performances:")
print(f"   - Précision sur test: {test_accuracy * 100:.2f}%")
print(f"   - Perte sur test: {test_loss:.4f}")
print(f"   - Epochs entraînés: {len(history.history['loss'])}")
print(f"\n💾 Fichiers générés:")
print(f"   - Modèle: {final_model_path}")
print(f"   - Courbes d'entraînement: {MODEL_DIR / 'training_curves.png'}")
print(f"   - Matrice de confusion: {MODEL_DIR / 'confusion_matrix.png'}")
print(f"   - Rapport de classification: {MODEL_DIR / 'classification_report.csv'}")
print(f"   - Métadonnées: {MODEL_DIR / 'model_metadata.json'}")
print("\n" + "="*70)

# Conclusion
if test_accuracy >= 0.9:
    print("\n🌟 Excellent! Le modèle atteint une précision supérieure à 90%!")
elif test_accuracy >= 0.8:
    print("\n✨ Très bien! Le modèle atteint une précision supérieure à 80%!")
elif test_accuracy >= 0.7:
    print("\n👍 Bien! Le modèle atteint une précision supérieure à 70%!")
else:
    print("\n⚠️ Le modèle pourrait être amélioré. Considérez:")
    print("   - Plus de données d'entraînement")
    print("   - Data augmentation plus aggressive")
    print("   - Architecture plus profonde")
    print("   - Transfer learning (ex: VGG16, ResNet)")